In [1]:
!pip install optuna

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import os

import torchvision
from torch import nn
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets
from torchvision.transforms import v2
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast


import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

from tqdm.notebook import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import optuna
from optuna.trial import TrialState

In [4]:
def define_model(trial):
    n_layersC = trial.suggest_int("n_layersC", 3, 6)
    image_size = 600
    layers = []
    in_features = 3

    for i in range(n_layersC):
        out_features = trial.suggest_int("n_neurons{}".format(i), 4, 128)
        kernel_size1 = 3
        stride = 1
        padding = 1

        layers.append(nn.Conv2d(in_channels=in_features, out_channels=out_features, kernel_size=kernel_size1, stride=stride, padding=padding))
        layers.append(nn.ReLU())
        layers.append(nn.AvgPool2d(kernel_size=2, stride=2))
        in_features = out_features

        image_size = (image_size + 2 * padding - kernel_size1) // stride + 1
        image_size = (image_size - 2) // 2 + 1
        if image_size <= 0:
            raise ValueError("Image size has become too small")

    layers.append(nn.Flatten())

    num_features = in_features * image_size * image_size

    n_layers = trial.suggest_int("n_layers", 2, 4)
    for i in range(n_layers):
        out_neurons = trial.suggest_int("n_units_l{}".format(i), 4, num_features)
        layers.append(nn.Linear(num_features, out_neurons))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        num_features = out_neurons

    layers.append(nn.Linear(num_features, 1))

    return nn.Sequential(*layers)

In [5]:
# class EarlyStopper:
#     def __init__(self, patience=1, min_delta=0):
#         self.patience = patience
#         self.min_delta = min_delta
#         self.counter = 0
#         self.min_validation_loss = float('inf')

#     def early_stop(self, validation_loss):
#         if validation_loss < self.min_validation_loss:
#             self.min_validation_loss = validation_loss
#             self.counter = 0
#         elif validation_loss > (self.min_validation_loss + self.min_delta):
#             self.counter += 1
#             if self.counter >= self.patience:
#                 return True
#         return False

In [6]:
transform = v2.Compose([
      v2.ToTensor(),
      v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform2 = v2.Compose([
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(90),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainset1 = datasets.ImageFolder(root="/content/drive/MyDrive/Курсовая 2 курс/data/training",
                                        transform=transform)
trainset2 = datasets.ImageFolder(root="/content/drive/MyDrive/Курсовая 2 курс/data/training",
                                        transform=transform2)
train_set = torch.utils.data.ConcatDataset([trainset1, trainset2])

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 16,
                                          shuffle=True, num_workers=2, drop_last = True)

validset = datasets.ImageFolder(root="/content/drive/MyDrive/Курсовая 2 курс/data/validation",
                                      transform=transform)
valid_loader = torch.utils.data.DataLoader(validset, batch_size = 16,
                                          shuffle=False, num_workers=2, drop_last = True)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

criterion = nn.BCEWithLogitsLoss()
total_step = len(train_loader)
num_epochs = 10

cuda


In [8]:
def objective(trial):

    model = define_model(trial).to(device)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)

    train_loss_list = []
    train_acc_list = []
    val_acc_list = []
    final_acc = []
    cum_loss = 0

    patience = 6
    best_loss = float('inf')
    epochs_no_improve = 0
    for epoch in tqdm(range(num_epochs)):
        # training
        for i, (images, labels) in enumerate(tqdm(train_loader)):
            if device.type == 'cuda':
                images = images.cuda()
                labels = labels.cuda()
            # feed forward
            outputs = model(images)
            labels = labels.unsqueeze(1).float()
            loss = criterion(outputs, labels)
            cum_loss += loss.item()
            # back propagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            # accuracy
            total = labels.size(0)
            predicted = (outputs.data > 0.5).float()
            correct = (predicted == labels).sum().item()
            if i % 16 == 15:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, cum_loss / 16,
                              (correct / total) * 100))
                cum_loss = 0
                train_loss_list.append(loss.item())
                train_acc_list.append(correct / total)

        # validation
        correct_count = 0
        total = 0
        validation_loss = 0.0
        for i, (images, labels) in enumerate(tqdm(valid_loader)):
            if device.type == 'cuda':
                images = images.cuda()
                labels = labels.cuda()
            # feed forward
            outputs = model(images)
            labels = labels.unsqueeze(1).float()
            predicted = (outputs.data > 0.5).float()
            correct = (predicted == labels).sum().item()
            total += labels.size(0)
            correct_count += correct
            validation_loss += criterion(outputs, labels).item() * labels.size(0)

        validation_loss /= total
        accuracy = correct_count / total

        trial.report(accuracy, epoch)
        print(f'Validated accuracy {accuracy}')
        if trial.should_prune():
            print('PRUNED')
            raise optuna.exceptions.TrialPruned()

        if validation_loss < best_loss:
            best_loss = validation_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping!")
                break
    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials = 1200, timeout = 6000)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-02-04 08:38:14,123] A new study created in memory with name: no-name-cc0f548e-ed9e-4787-a287-b9498945bb5a
<ipython-input-8-399d5edb69bd>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/230 [00:00<?, ?it/s]

Epoch [1/10], Step [16/230], Loss: 0.6930, Accuracy: 37.50%
Epoch [1/10], Step [32/230], Loss: 0.6925, Accuracy: 56.25%
Epoch [1/10], Step [48/230], Loss: 0.6927, Accuracy: 62.50%
Epoch [1/10], Step [64/230], Loss: 0.6924, Accuracy: 50.00%
Epoch [1/10], Step [80/230], Loss: 0.6921, Accuracy: 50.00%
Epoch [1/10], Step [96/230], Loss: 0.6912, Accuracy: 25.00%
Epoch [1/10], Step [112/230], Loss: 0.6921, Accuracy: 43.75%
Epoch [1/10], Step [128/230], Loss: 0.6911, Accuracy: 25.00%
Epoch [1/10], Step [144/230], Loss: 0.6938, Accuracy: 56.25%
Epoch [1/10], Step [160/230], Loss: 0.6935, Accuracy: 43.75%
Epoch [1/10], Step [176/230], Loss: 0.6929, Accuracy: 68.75%
Epoch [1/10], Step [192/230], Loss: 0.6914, Accuracy: 56.25%
Epoch [1/10], Step [208/230], Loss: 0.6933, Accuracy: 31.25%
Epoch [1/10], Step [224/230], Loss: 0.6921, Accuracy: 56.25%


  0%|          | 0/14 [00:00<?, ?it/s]

Validated accuracy 0.47767857142857145


  0%|          | 0/230 [00:00<?, ?it/s]

Epoch [2/10], Step [16/230], Loss: 0.9523, Accuracy: 25.00%
Epoch [2/10], Step [32/230], Loss: 0.6929, Accuracy: 25.00%
Epoch [2/10], Step [48/230], Loss: 0.6924, Accuracy: 50.00%
Epoch [2/10], Step [64/230], Loss: 0.6924, Accuracy: 31.25%
Epoch [2/10], Step [80/230], Loss: 0.6938, Accuracy: 31.25%
Epoch [2/10], Step [96/230], Loss: 0.6934, Accuracy: 43.75%
Epoch [2/10], Step [112/230], Loss: 0.6922, Accuracy: 50.00%
Epoch [2/10], Step [128/230], Loss: 0.6929, Accuracy: 62.50%
Epoch [2/10], Step [144/230], Loss: 0.6929, Accuracy: 37.50%
Epoch [2/10], Step [160/230], Loss: 0.6938, Accuracy: 75.00%
Epoch [2/10], Step [176/230], Loss: 0.6928, Accuracy: 43.75%
Epoch [2/10], Step [192/230], Loss: 0.6923, Accuracy: 12.50%
Epoch [2/10], Step [208/230], Loss: 0.6926, Accuracy: 37.50%
